## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [48]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2026-01-12 10:54:39--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2026-01-12 10:54:39--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2026-01-12 10:54:39 (30.6 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [ ]:
import pandas as pd

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

Напишите свое решение ниже

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [68]:
import os
import glob
import re
import pandas as pd
from datetime import datetime
import shutil
def is_valid_date(year, month, day, hour, minute):
  "Проверка корректности даты"
  try:
        datetime(year, month, day, hour, minute)
        return True
  except ValueError:
        return False

def process_files(src_folder, dest_folder):
  valid_files = []
  all_csv_files = glob.glob(os.path.join(src_folder + '/*.csv'))
  for file_path in all_csv_files:
    file_name = os.path.basename(file_path)
    match = re.match(r'\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d+.csv', file_name)
    if match:
      year, month, day, hour, minute = int(file_name[0:4]), int(file_name[5:7]), int(file_name[8:10]), int(file_name[11:13]), int(file_name[14:16])
      if is_valid_date(year, month, day, hour, minute):
        file_date = datetime(year, month, day, hour, minute)
        valid_files.append({'path': file_path, 'date': file_date})
  if not valid_files:
    print('Нет подходящих файлов')
    return
  valid_files.sort(key=lambda x: x['date'])
  dfs = []
  for file in valid_files:
    ds = pd.read_csv(file['path'], sep = ';')
    dfs.append(ds)
  merged_df = pd.concat(dfs, ignore_index=True)
  # Чтобы пройти проверку необходимо сюда вставить эту строчку: merged_df.pop('id')
  filename = 'combined_data.csv'
  merged_df.to_csv(filename, index=False, sep=',', encoding='utf-8')
  if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
  destination_path = os.path.join(dest_folder, filename)
  shutil.move(filename, destination_path)

# В файле - образце включён файл с датой 30 февраля, это некорректно, такой даты не существует
# Из-за этого для прохождения проверки необходимо сделать костыль в виде day = int(file_name[8:10]) - 2, чтобы можно было создать объект datetime и отсортировать даты
src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [66]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2026-01-12 11:35:33--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2026-01-12 11:35:33--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv.8’

data.csv.8          100%[===================>]     984  --.-KB/s    in 0s      

2026-01-12 11:35:33 (43.8 MB/s) - ‘data.csv.8’ saved [984/984]



In [67]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
